<a href="https://colab.research.google.com/github/JunHyeongKim73/2019-2-OSS-L2/blob/gh-pages/DTHON_KFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=9e06438d2dbf029febfe6e421eab73480433cad4e341f6515983c4a565b07c06
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


# Import Libraries

In [3]:
import numpy as np
import torch
from matplotlib import pyplot as plt
from torchsummary import summary

import os
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import time
import copy

from torch.optim import lr_scheduler
from torchvision import datasets
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader

from efficientnet_pytorch import EfficientNet
from tqdm.auto import tqdm
from sklearn.model_selection import KFold
from torchvision import models

# Data Transformer Class

In [4]:
class DataTransformer():
    def __init__(self, dataset, transforms):
        self.dataset = dataset
        self.transforms = transforms
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, index):
        image, label = self.dataset[index]
        if self.transforms:
            image = self.transforms(image)

        return image, label

# Image Augmentation

In [5]:
train_transforms = transforms.Compose([
    transforms.Resize(224),
    # transforms.Grayscale(num_output_channels=3),
    # transforms.RandomRotation(30),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])                            
])

test_transforms = transforms.Compose([
    transforms.Resize(224),
    # transforms.Grayscale(num_output_channels=3),
    # transforms.RandomRotation(30),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])                            
])

# Load Dataset

In [6]:
dataset = datasets.CIFAR10(root='./cifar10_data/', train=True, download=True)
dataset, _ = torch.utils.data.random_split(dataset, [10000, 40000])

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar10_data/cifar-10-python.tar.gz to ./cifar10_data/


# EfficientNet Model Class


In [12]:
class EffNetModel(nn.Module):
    def __init__(self, model_name):
        super(EffNetModel, self).__init__()
        
        self.backbone = EfficientNet.from_pretrained(model_name)
        self.out_features = self.backbone._fc.out_features
        self.fc = nn.Linear(self.out_features, 10)

        #nn.init.xavier_normal_(self.fc.weight)

    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(x)
        x = torch.sigmoid(x)

        return x

# ResNet Model Class

In [8]:
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()

        self.backbone = models.resnet18(pretrained=True)

        self.fc = nn.Linear(1000, 10)

        #nn.init.xavier_normal_(self.fc.weight)

    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(x)

        return x

# Train Model

In [ ]:
batch_size = 16
epochs = 40

kfold = KFold(n_splits=5, shuffle=True, random_state=0)
# K-Fold Cross Validation
best_models = [] # 폴드별로 가장 acc가 높은 모델 저장
for fold_idx, (train_idx, test_idx) in enumerate(kfold.split(dataset), 1):
    # Train & Test Dataset 정의
    train_subset = [dataset[idx] for idx in train_idx]
    test_subset =[dataset[idx] for idx in test_idx]

    train_dataset = DataTransformer(train_subset, train_transforms)
    test_dataset = DataTransformer(test_subset, test_transforms)
    # DataLoader 정의
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    # Load Model
    model_name = 'efficientnet-b0'

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = EffNetModel(model_name)
    #model = ResNetModel()
    model.to(device)

    # Hyper Parameter
    learning_rate = 0.001
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    scheduler = lr_scheduler.StepLR(optimizer=optimizer, step_size=7, gamma=0.1)

    # Initialize Variables
    best_acc = 0.0
    best_loss = 0.0
    best_epoch = 0
    best_model = None

    early_stop_cnt = 0
    early_stop = 7
    min_loss = 10^3

    since = time.time()

    for epoch in range(epochs):
        epoch += 1
        print('Epoch {}/{}'.format(epoch, epochs))
        print('-' * 10)
        
        model.train()

        train_len = 0    
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in tqdm(train_loader, desc='Train'):
            inputs = inputs.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training 'test
                loss.backward()
                optimizer.step()

            # statistics
            batch_size = inputs.size(0)
            running_loss += loss.item() * batch_size
            running_corrects += torch.sum(preds == labels.data)

            train_len += batch_size
        
        scheduler.step()

        epoch_loss = running_loss / train_len
        epoch_acc = running_corrects.double() / train_len

        print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    
        model.eval()   # Set model to evaluate mode

        test_len = 0
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in tqdm(test_loader, desc='Test'):
            inputs = inputs.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            # track history if only in train
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

            # statistics
            batch_size = inputs.size(0)
            running_loss += loss.item() * batch_size
            running_corrects += torch.sum(preds == labels.data)

            test_len += batch_size

        epoch_loss = running_loss / test_len
        epoch_acc = running_corrects.double() / test_len

        print('Test Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        # Save the Best Model Information
        if epoch_acc > best_acc:
            early_stop_cnt = 0
            best_acc = epoch_acc
            best_loss = epoch_loss
            best_epoch = epoch
            best_model = model
        
        elif epoch_loss < min_loss:
            early_stop_cnt = 0
            min_loss = epoch_loss
        
        else:
            early_stop_cnt += 1
            
        if early_stop_cnt > early_stop:
            print('Early Stopped!!')
            break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # Save the Best Model
    PATH = "/content/drive/MyDrive/models/"
    torch.save(best_model, f'{PATH}{fold_idx}_{model_name}_{best_acc}_{best_loss}_epoch_{best_epoch}.pth')
    # 폴드별로 가장 좋은 모델 저장
    best_models.append(best_model)
    # 현재 1 Fold로만 진행
    break

Loaded pretrained weights for efficientnet-b0
Epoch 1/40
----------


Train:   0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
  !nvidia-smi

Tue Nov  2 08:26:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------